## g++ compile command

* `-g`: turn on debugging (so GDB gives more friendly output)
* `-c`: compile the source code into an object file (`.o`)
* `-o`: specify the output file name
* `-O` or `-O2`: turn on the optimization
* `-Wall`: turn on most warnings
* `-I<include path>`: specify an include directory
* `-L<library path>`: specify a lib directory
* `-l<library>`: link with library lib<library>.a

## compilation notes

### build a dynamic library

Suppose `a.cc` and `b.cc` are the source code to be built into one library and link it to main.cc,
besides the main.cc should contain interface files, like `a.h` and `b.h`, respectively, one need to following the procedure to create a library [detailes check here](https://renenyffenegger.ch/notes/development/languages/C-C-plus-plus/GCC/create-libraries/index):

```makefile
#generate the path independent object files
g++ a.o -c -fpic a.cc
g++ b.o -c -fpic b.cc 

#compile the binaries into dynamic library
g++ -shared a.o b.o -o libname.so

#compile the main into .o
g++ -c main.cc -o main.o

#linking the library libname.so (refered as -lname in compiler option) to the file to make the executable "program"
#It supposed that name of library started with lib and object ended with .o, so the "lib" and ".o" shouldn't appear
g++ program -o main.o -L /path/to/the/dynamic/library/ -lname 
```

### build a static library
The executable compiled output is independent on any .a or .o so that it could be distriubted

```makefile
#compile the object files (no need to be pic)
g++ a.o -c a.cc
g++ b.o -c b.cc 

ar rcs /path/to/output/libname.a /path/to/source/a/a.o /path/to/source/b/b.o

#link the static library, (I assume -o should be followed by the path the output, but not sure. Test is needed)
g++ main.o -L/path/to/the/library -lname -o /path/to/output
```

Some useful compile option:
```
-rpath = dir : 
```
Add a directory to the runtime library search path. This is used when linking an ELF executable with shared objects. All -rpath arguments are concatenated and passed to the runtime linker, which uses them to locate shared objects at runtime. The `-rpath` option is also used when locating shared objects which are needed by shared objects explicitly included in the link; **this is very useful in case you want to call your compiled executable in bash on MacOS, since the new MacOS will clean the variable `DYLD_LIBRARY_PATH` (actually, all the executable from `/usr/bin` will clean this varaible) which will leads to the crash. The way to solve it is to add the option for the run time path by using `-rpath`:**
```
-Wl,-rpath,<root-installation-prefix>/lib
```

## compilation by using makefile

To use the 

## C++ notes

#### function pointer

```cpp
void my_int_func(int x){
    printf("%d\n", x);
}

int main(){
    void (*foo)(int); // func pointer declartion
    foo = &my_int_func; //sign the pointer to the function
    (*foo)(2); // call the function
    return 0;
}
```

#### iterator loop example

```cpp
//very compact loop control, direct access to the current element.
for (auto& elem: v) {
     // if the current index is needed:
     auto i = &elem - &v[0];

    // any code including continue, break, return
}
```

#### virtual function

```cpp
// the virtual functoin will be overloaded everywhere in the inherit class, even it has been called in the parent class function
class a {
        public : a(){};
                 void call(){run();}
                 virtual void run(){cout<<"a"<<endl;}
};

class b : public a {
        public : b(){};
                 virtual void run(){cout<<"b"<<endl;}
};

void check(){
        b ss;
        ss.call();// it will cout b which shows the virtual function has been overloaded.
        ss.a::run(); // call the function coming from parent scope.
}
```

### operators

* [Operator precedence](https://en.cppreference.com/w/cpp/language/operator_precedence)

* The overloadable operators:

`+   -   *   /   %    ^    &   |    ~`
`!   =   <   >   +=   -=   *=  /=   %=`
`^=  &=  |=  <<  >>   <<=  >>= ==   !=`
`<=  >=  &&  ||  ++   --   ,   ->*  ->`




## Makefile
[More infomation](http://nuclear.mutantstargoat.com/articles/make/)
Suppose we are going to compile a dynamic library using makefile and some conditions listed here:
1. the interface and soruce codes are in `pwd/interface/` and `pwd/src/` respectively. 
2. Put all the `.o` object files in the `pwd/bin/`
3. Use all the object files in `pwd/bin/` to build the dynamic library `pwd/lib/libedm.so` 

The example makefile is:

```csh
CC = g++
INCLUDE= -I $(PWD)/interface/ -I $(PWD)/../plugin/
#will take all the files in src/ ended .cc as input for the variable SRCS
SRCS:= $(wildcard src/*.cc)
#substitue all the src/*.cc files as bin/*.o
BINS:= $(SRCS:src/%.cc=bin/%.o)

#default targets, run over all variables in BINS and libedm.so
all: $(BINS) libedm.so

#make the rule for each bin/*.o files: using src/*.cc files to compile the object files, the output is bin/*.o which specified by the word $@

bin/%.o: src/%.cc
    @echo "=====  Creating object " $@ "=========="
    $(CC) -c $< -o $@ -fpic $(INCLUDE) $(ROOT)

#again, the wildcard will take all the files bin/*.o as input to compile libedm.so
libedm.so: $(wildcard bin/*.o)
    $(CC) -shared bin/rootEDM.o -o ../lib/libedm.so $(INCLUDE)

#make a phony rule, no input and output rules for clean or any other type of command
.PHONY: clean

clean:
    rm -rf bin/*.o ./*.*~ src/*.*~
```